In [ ]:
# # Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def transform_date(date):
  components = date.split('_')
  month = {"january": "01", "february": "02", "march": "03", "april": "04", "may": "05", "june": "06", "july": "07", "august": "08", "september": "09", "october": "10", "november": "11", "december": "12"}
  if len(components[2])==1:
    components[2] = '0' + components[2]
  return components[0] + month[components[1].lower()] + components[2]

#Number of views

In [ ]:
# #number of views
# https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Foo/daily/20151010/20151012
# https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews

In [ ]:
!pip install -q git+https://github.com/Commonists/pageview-api.git

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install -q swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 KB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.9 MB/s eta 0:00:00


In [ ]:
from numpy import around
import pageviewapi
from datetime import datetime, timedelta
import swifter
import pandas as pd


# https://github.com/Comhttps://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&titles=2021%E2%80%932022_Boulder_County_fires&rvprop=ids&rvlimit=1&rvstart=2022-01-01T00%3A00%3A00.010Zmonists/pageview-api
def get_n_views(title, from_, to_):
  try:
    res = sum([i['views'] for i in pageviewapi.per_article('en.wikipedia', title, from_, to_,
                        access='all-access', agent='all-agents', granularity='daily')['items']])
  except:
    #print(title, from_, to_)
    res = -1
  return res
  
def get_views_features(title, day):
  before_2 = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=2), '%Y%m%d')
  after_2 = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=2), '%Y%m%d')
  before_6 = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=6), '%Y%m%d')

  around = get_n_views(title, before_2, after_2)
  before = get_n_views(title, before_6, before_2)

  boost = around/before

  prev_day = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=1), '%Y%m%d')
  week_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=7), '%Y%m%d')
  month_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=30), '%Y%m%d')
  year_before = datetime.strftime(datetime.strptime(day, '%Y%m%d') - timedelta(days=365), '%Y%m%d')

  day_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=1), '%Y%m%d')
  week_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=7), '%Y%m%d')
  month_after = datetime.strftime(datetime.strptime(day, '%Y%m%d') + timedelta(days=30), '%Y%m%d')

  same_day_views = get_n_views(title, day, day)
  prev_day_views = get_n_views(title, prev_day, day)
  week_before_views = get_n_views(title, week_before, day)
  month_before_views = get_n_views(title, month_before, day)
  year_before_views = get_n_views(title, year_before, day)

  day_after_views = get_n_views(title, day, day_after)
  week_after_views = get_n_views(title, day, week_after)
  month_after_views = get_n_views(title, day, month_after)

  return [around, before, boost, same_day_views, prev_day_views, week_before_views, month_before_views, 
          year_before_views, day_after_views, week_after_views, month_after_views]


Current events 2022

In [ ]:
df = pd.read_csv("drive/My Drive/Diploma/second_try/Code/events_2022.csv")

df['title'] = df['Title link'].str.split('/', expand=True)[2]
df['date'] = df['Date'].apply(lambda x: transform_date(x))
df.head()

,Unnamed: 0,Date,Category,Text,Title link,Articles,Ext links,HTML,title,date
0,0,2022_January_1,Armed conflicts and attacks\n,Haitian Prime Minister Ariel Henry survives an...,/wiki/Prime_Minister_of_Haiti,"['Prime Minister of Haiti', 'Ariel Henry', 'Go...",['https://www.bbc.com/news/world-latin-america...,"<li><a href=""/wiki/Prime_Minister_of_Haiti"" ti...",Prime_Minister_of_Haiti,20220101
1,1,2022_January_1,Disasters and accidents\n,2021–2022 Boulder County fires\nAt least three...,/wiki/2021%E2%80%932022_Boulder_County_fires,"['2021–2022 Boulder County fires', 'Boulder Co...",['https://www.cnn.com/2022/01/01/us/colorado-w...,"<li><a class=""mw-redirect"" href=""/wiki/2021%E2...",2021%E2%80%932022_Boulder_County_fires,20220101
2,2,2022_January_1,Disasters and accidents\n,"At least three people are confirmed missing, a...","/wiki/Boulder_County,_Colorado","['Boulder County, Colorado', 'United States']",['https://www.cnn.com/2022/01/01/us/colorado-w...,<li>At least three people are confirmed missin...,"Boulder_County,_Colorado",20220101
3,3,2022_January_1,Disasters and accidents\n,Vaishno Devi Temple stampede\nTwelve people ar...,/wiki/Vaishno_Devi_Temple_stampede,"['Vaishno Devi Temple stampede', 'Stampede', '...",['https://www.aljazeera.com/news/2022/1/1/12-d...,"<li><a href=""/wiki/Vaishno_Devi_Temple_stamped...",Vaishno_Devi_Temple_stampede,20220101
4,4,2022_January_1,Disasters and accidents\n,Twelve people are killed and 13 others injured...,/wiki/Stampede#Human_stampedes,"['Stampede', 'Vaishno Devi Temple', 'India', '...",['https://www.aljazeera.com/news/2022/1/1/12-d...,<li>Twelve people are killed and 13 others inj...,Stampede#Human_stampedes,20220101


In [ ]:
views_features = df.swifter.apply(lambda x: get_views_features(x['title'], x['date']), axis=1)

Pandas Apply:   0%|          | 0/9800 [00:00<?, ?it/s]

In [ ]:
views_features_df = pd.DataFrame(views_features.tolist(), columns=["around", "before", "boost", "same_day_views", "prev_day_views", "week_before_views", "month_before_views", 
          "year_before_views", "day_after_views", "week_after_views", "month_after_views"])

In [ ]:
views_features_df.to_csv('views_features_df_2022.csv') 
files.download('views_features_df_2022.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
views_features_df.shape

(9800, 11)

In [ ]:
df.loc[1400]

page_title    1923_Liverpool_City_Council_election
page_id                                   49372580
date                                      20220601
count_rev                                        1
Name: 1400, dtype: object

In [ ]:
pageviewapi.per_article('en.wikipedia', '1923_Liverpool_City_Council_election', '20221106', '20221120',
                        access='all-access', agent='all-agents', granularity='daily')

AttrDict({'items': [{'project': 'en.wikipedia', 'article': '1923_Liverpool_City_Council_election', 'granularity': 'daily', 'timestamp': '2022110600', 'access': 'all-access', 'agent': 'all-agents', 'views': 1}, {'project': 'en.wikipedia', 'article': '1923_Liverpool_City_Council_election', 'granularity': 'daily', 'timestamp': '2022110700', 'access': 'all-access', 'agent': 'all-agents', 'views': 1}, {'project': 'en.wikipedia', 'article': '1923_Liverpool_City_Council_election', 'granularity': 'daily', 'timestamp': '2022110900', 'access': 'all-access', 'agent': 'all-agents', 'views': 2}, {'project': 'en.wikipedia', 'article': '1923_Liverpool_City_Council_election', 'granularity': 'daily', 'timestamp': '2022111000', 'access': 'all-access', 'agent': 'all-agents', 'views': 3}, {'project': 'en.wikipedia', 'article': '1923_Liverpool_City_Council_election', 'granularity': 'daily', 'timestamp': '2022111100', 'access': 'all-access', 'agent': 'all-agents', 'views': 3}, {'project': 'en.wikipedia', 'a